In [1]:
import nltk
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from flask import Flask, render_template, request
app = Flask(__name__,template_folder='template')


In [3]:
f=open('final.txt','r',errors = 'ignore')
raw = f.read()
sent_tokens = nltk.sent_tokenize(raw)

In [4]:
Greeting_Inputs = ("hello", "hi", "greetings", "sup", "good morning","hey","good evening","good afternoon")
Greeting_Responses = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in Greeting_Inputs:
            return random.choice(Greeting_Responses)


In [5]:
generic_response = ["My sole purpose is to talk. I am neither happy, sad or sleepy. What about you?", "Well,You already know the answer. Dont'you? "]

In [6]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [7]:
def response(user_response):
    bot_response=''
    sent_tokens.append(user_response)
    TfidfVector = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVector.fit_transform(sent_tokens)
    value = cosine_similarity(tfidf[-1], tfidf)
    final =value.argsort()[0][-2]
    flat = value.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        bot_response=bot_response+"Rephrase it into simpler words!"
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[final]
        return bot_response

In [8]:
def chat(user_response):
    user_response=user_response.lower()
    keyword = "wisdom"
    keyword1 = "quote"
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            return "You are welcome"
        else:
            if(greeting(user_response)!=None):
                return greeting(user_response)
            elif(user_response.find("how are you") != -1 or user_response.find("do you feel") != -1 or user_response.find("do you have emotions")!=-1 or user_response.find("do you sleep")!=-1):
                return random.choice(generic_response)
            elif(user_response.find("what is your name")!=-1):
                return "You can call me anything you want! I prefer Bot:)What is your name?"
            elif(user_response.find(keyword)!=-1 or user_response.find(keyword1)!=-1):
                return "We've all got both light and dark inside us. What matters is the part we choose to act on. That's who we really are!"
            else:
                return (response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        return "Bye! It was an interesting conversation."


In [9]:
@app.route("/")
def home():
    return render_template("index.html")

In [10]:
@app.route("/get")   
def get_bot_response():
    userText = request.args.get('msg')
    return chat(userText)


In [11]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Sep/2020 17:19:55] "GET / HTTP/1.1" 200 -
